In [29]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain import LLMChain
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

chat = ChatOpenAI(
    openai_api_key=api_key,
    temperature=0.1,
    streaming=True,
    callbacks=[
       StreamingStdOutCallbackHandler(),
    ],
)

# FewShot용 예시
examples = [
    {
        "movie_name": "Inception",
        "director": "크리스토퍼 놀란",
        "cast": "레오나르도 디카프리오, 조셉 고든 레빗, 엘렌 페이지",
        "budget": "160,000,000 달러",
        "revenue": "829,895,144 달러",
        "genre": "SF, 스릴러",
        "synopsis": "꿈을 통해 정보를 훔치는 산업 스파이들이 가장 깊은 무의식을 탐험하는 이야기"
    },
    {
        "movie_name": "Parasite",
        "director": "봉준호",
        "cast": "송강호, 이선균, 조여정, 최우식, 박소담",
        "budget": "11,000,000 달러",
        "revenue": "258,000,000 달러",
        "genre": "드라마, 스릴러",
        "synopsis": "가난한 가족과 부유한 가족이 얽히며 벌어지는 기묘하고 비극적인 사건"
    }
]

example_message_template = """
영화 이름: {movie_name}
감독: {director}
주요 출연진: {cast}
예산: {budget}
흥행 수익: {revenue}
장르: {genre}
간단한 시놉시스: {synopsis}
""".strip()

example_prompt = PromptTemplate(
    input_variables=["movie_name", "director", "cast", "budget", "revenue", "genre", "synopsis"],
    template=example_message_template
)

# FewShotPromptTemplate 구성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="아래는 영화 정보에 대한 예시입니다. 각 예시는 동일한 형식을 유지합니다.",
    suffix="\n이제 아래의 영화에 대해 같은 형식으로 답변하세요.\n\n영화 이름: {input}",
    input_variables=["input"]
)

# 테스트용 영화 이름
test_movie = "Interstellar"

chain = few_shot_prompt | chat

# chain.invoke() 결과를 변수에 담은 후 content만 출력
response = chain.invoke({"input": test_movie})

print("\n---최종 응답---")
print(response.content.strip())



---최종 응답---
감독: 크리스토퍼 놀란
주요 출연진: 매튜 맥커너히, 앤 해서웨이, 제시카 차스테인
예산: 165,000,000 달러
흥행 수익: 677,471,941 달러
장르: SF, 드라마
간단한 시놉시스: 지구가 멸망하기 전, 인류의 생존을 위해 우주 여행을 떠나는 우주 탐험가들의 이야기
